# Homogeneity
Calculates various clustering evaluation metrics.

Moreover an own measure was devleoped but not used for the thesis in the end.
An own measure was developed to achieve this is the number of pixels per cluster (ppc) in the prediction (as a fraction of the numbers of pixels in the original image).
Actually it is just another interpretation of the number of clusters that are detected within the whole dataset.
The ppc ground truth ratio (ppc-gtr) is the ppc of the prediction compared to the ppc of the original images and it gives an impression of the homogeneity of the result compared to the ground truth. 
Its drawback is, that it does not reflect how well the predictions perform regarding their correctness, but just how similarly the number of clusters is distributed. 



In [1]:
# basic imports
import random
import numpy as np
import os
import sys
import cv2
from collections import Counter


from tqdm import tqdm
# DL library imports
import torch
from torch.utils.data import Dataset
import segmentation_models_pytorch as smp
from sklearn.metrics import homogeneity_score, completeness_score
from sklearn import metrics
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import variation_of_information as vari
from skimage.metrics import adapted_rand_error
from scipy.ndimage import label as count_clusters
# import torch.nn as nn
# import torch.nn.functional as F

# libraries for loading image, plotting 
# import cv2
import matplotlib.pyplot as plt
import pandas as pd

sys.path.append('..')
from utils import IoU                  # metric class
from utils import evaluate_model, train_id_to_color

import sys
# from models import UNet, UNetEncoder, UNetDecoder

In [2]:
# load model
# MODEL1_PATH = os.path.join(model_dir, model1_name)
# MODEL1_PATH = '/home/j/j_sten07/thesis_test/output/unet_50ep_512px_train-10_randomsplit_lr3e-5/unet_50ep_512px_train-10_randomsplit_lr3e-5/unet_50ep_512px_train-10_randomsplit_lr3e-5_best.pt'
# MODEL1_PATH = '/home/j/j_sten07/thesis_test/output/unet_50ep_512px_train-5_14_randomsplit_lr3e-5/unet_50ep_512px_train-5_14_randomsplit_lr3e-5_best.pt'
# MODEL1_PATH = '/home/j/j_sten07/thesis_test/output/floodnet_test_crossentropy/floodnet_test_crossentropy_last.pt'
# MODEL1_PATH = '/home/j/j_sten07/thesis_test/output/unet_50ep_512px_train-10_randomsplit_lr3e-5/unet_50ep_512px_train-10_randomsplit_lr3e-5/unet_50ep_512px_train-10_randomsplit_lr3e-5_best.pt'

# MODEL1_PATH = '/palma/scratch/tmp/j_sten07/output/unet_150ep_1024px_test-14-15_randomsplit_lr5e-4_jaccard/unet_150ep_1024px_test-14-15_randomsplit_lr5e-4_jaccard_best.pt'
# MODEL1_PATH = '/palma/scratch/tmp/j_sten07/output/unet_100ep_512px_irrg-test-15_randomsplit_lr5e-4/unet_100ep_512px_irrg-test-15_randomsplit_lr5e-4_best.pt'


# MODEL2_PATH = os.path.join(model_dir, model2_name)
# MODEL2_PATH = '/home/j/j_sten07/thesis_test/output/floodnet_test_weightedCEnotnormalized/floodnet_test_weightedCEnotnormalized_last.pt'
# MODEL2_PATH = '/home/j/j_sten07/thesis_test/output/segformer_50ep_512px_train-10_randomsplit_lr3e-5/segformer_50ep_512px_train-10_randomsplit_lr3e-5/segformer_50ep_512px_train-10_randomsplit_lr3e-5_best.pt'
# MODEL2_PATH = '/home/j/j_sten07/thesis_test/output/segformer_50ep_512px_train-10_randomsplit_lr3e-5/segformer_50ep_512px_train-10_randomsplit_lr3e-5/segformer_50ep_512px_train-10_randomsplit_lr3e-5_best.pt'
# MODEL2_PATH = '/home/j/j_sten07/thesis_test/output/segformer_50ep_512px_train-5_14_randomsplit_lr3e-5/segformer_50ep_512px_train-5_14_randomsplit_lr3e-5_best.pt'

# MODEL2_PATH = '/palma/scratch/tmp/j_sten07/output/segformer_100ep_1024px_test-14-15_randomsplit_lr4e-5/segformer_100ep_1024px_test-14-15_randomsplit_lr4e-5_best.pt'
# MODEL2_PATH = '/palma/scratch/tmp/j_sten07/output/segformer_100ep_512px_irrg-test-15_randomsplit_lr5e-5/segformer_100ep_512px_irrg-test-15_randomsplit_lr5e-5_best.pt'


# patch_size = 1024
# dataset = 'potsdam'
# data_path= '/palma/scratch/tmp/j_sten07/data/Potsdam/patches/1024px/test_14_15'
# data_path = '/home/j/j_sten07/thesis_test/data/Potsdam/patches/512px/ir_test_15'

#----------------------------------------------------------
#---------------- JupyterHub ------------------------------
# #----------------------------------------------------------
# models_dir = '/home/j/j_sten07/thesis_test/output/'

# # #----------------------------------------------------------
# patch_size = 512
# label_add = 'S'
# dataset = 'potsdam'
# data_path= '/palma/scratch/tmp/j_sten07/data/Potsdam/patches/512px/test_14_15'  # use same test data as for other models
# # model1_name = 'unet_train-10_ps512px_400ep_lr5e-5_jaccard'
# # model2_name = 'segformer_train-10_ps512px_400ep_lr5e-5_jaccard'
# # norm_dataset = 'own'
# model1_name = 'unet_train-10_ps512px_400ep_lr5e-5_jaccard_ownNorm'
# model2_name = 'segformer_train-10_ps512px_400ep_lr5e-5_jaccard_ownNorm'
# norm_dataset = 'potsdam'



#----------------------------------------------------------
#---------------- PALMA -----------------------------------
#----------------------------------------------------------
models_dir = '/palma/scratch/tmp/j_sten07/output/'

# #----------------------------------------------------------
# patch_size = 1024
# label_add = '1024'
# dataset = 'potsdam'
# data_path= '/palma/scratch/tmp/j_sten07/data/Potsdam/patches/1024px/test_14_15'
# # model1_name = 'unet_100ep_ps1024_test-14-15_randomsplit_lr5e-4_jaccard_b2w2'
# # model2_name = 'segformer_100ep_ps1024_test-14-15_randomsplit_lr5e-5_jaccard_b2w2'
# # norm_dataset = 'own'
# model1_name = 'unet_400ep_ps1024_test-14-15_randomsplit_lr5e-5_jaccard_b2w2_ownNorm'
# model2_name = 'segformer_400ep_ps1024_test-14-15_randomsplit_lr5e-5_jaccard_b2w2_ownNorm'
# norm_dataset = 'potsdam'

#----------------------------------------------------------
# patch_size = 512
# label_add = '512'
# dataset = 'potsdam'
# data_path= '/palma/scratch/tmp/j_sten07/data/Potsdam/patches/512px/test_14_15' # data_path= '/home/j/j_sten07/thesis_test/data/Potsdam/patches/512px/test_14_15'  # if PALMA doesnt work
# # model1_name = 'unet_200ep_512px_test-14-15_randomsplit_lr5e-4_jaccard'
# # model2_name = 'segformer_200ep_512px_test-14-15_randomsplit_lr6e-5_jaccard'
# # norm_dataset = 'own'
# model1_name = 'unet_300ep_512px_test-14-15_randomsplit_lr5e-5_jaccard_ownNorm'
# model2_name = 'segformer_300ep_ps512_test-14-15_randomsplit_lr5e-5_jaccard_ownNorm'
# norm_dataset = 'potsdam'


# #----------------------------------------------------------
# patch_size = 256
# label_add = '256'
# dataset = 'potsdam'
# data_path= '/palma/scratch/tmp/j_sten07/data/Potsdam/patches/256px/test_14_15' # data_path= '/home/j/j_sten07/thesis_test/data/Potsdam/patches/512px/test_14_15'  # if PALMA doesnt work
# # model1_name = 'unet_200ep_ps256_test-14-15_randomsplit_lr5e-4_jaccard'
# # model2_name = 'segformer_200ep_ps256_test-14-15_randomsplit_lr6e-5_jaccard'
# # norm_dataset = 'own'
# model1_name = 'unet_200ep_ps256_test-14-15_randomsplit_lr5e-5_jaccard_ownNorm'
# model2_name = 'segformer_200ep_ps256_test-14-15_randomsplit_lr5e-5_jaccard_ownNorm'
# norm_dataset = 'potsdam'

# #----------------------------------------------------------
# patch_size = 128
# label_add = '128'
# dataset = 'potsdam'
# data_path= '/palma/scratch/tmp/j_sten07/data/Potsdam/patches/128px/test_14_15'
# # maybe these will still be changed
# model1_name = 'unet_300ep_ps128_test-14-15_randomsplit_lr3e-5_jaccard_ownNorm' 
# model2_name = 'segformer_300ep_ps128_test-14-15_randomsplit_lr3e-5_jaccard_ownNorm' 
# norm_dataset = 'potsdam'


#----------------------------------------------------------
#------------------- IRRG ---------------------------------
# patch_size = 512
# label_add = 'IR'
# dataset = 'potsdam'
# # data_path = '/palma/scratch/tmp/j_sten07/data/Vaihingen/patches/256px'
# # norm_dataset = 'vaihingen'
# data_path= '/palma/scratch/tmp/j_sten07/data/Potsdam/patches/512px/ir_test_14_15'
# norm_dataset = 'potsdam_irrg'
# # model1_name = 'unet_200ep_ps512px-irrg_ir-test-14-15_randomsplit_lr5e-4_jaccard'
# # model2_name = 'segformer_200ep_ps512-irrg_ir-test-14-15_randomsplit_lr6e-5_jaccard'
# # norm_dataset = 'own'
# model1_name = 'unet_300ep_ps512px-irrg_ir-test-14-15_randomsplit_lr5e-5_jaccard_ownnorm'
# model2_name = 'segformer_300ep_ps512px-irrg_ir-test-14-15_randomsplit_lr5e-5_jaccard_ownnorm'



# ----------------------------------------------------------
# ------------- Floodnet -----------------------------------
# patch_size = 512
# dataset = 'floodnet'
# label_add = 'floodnet-512'
# data_path= '/palma/scratch/tmp/j_sten07/data/FloodNet/512px' # data_path= '/home/j/j_sten07/thesis_test/data/Potsdam/patches/512px/test_14_15'  # if PALMA doesnt work
# model1_name = 'unet_200ep_floodnet-ps512_randomsplit_lr3e-5_weightedCE_b2'
# model2_name = 'segformer_200ep_floodnet-ps512_randomsplit_lr3e-5_weightedCE_b2'
# norm_dataset = 'floodnet'


# #----------------------------------------------------------
patch_size = 1024
dataset = 'floodnet'
label_add = 'floodnet-1024'
data_path= '/palma/scratch/tmp/j_sten07/data/FloodNet/1024px' # data_path= '/home/j/j_sten07/thesis_test/data/Potsdam/patches/512px/test_14_15'  # if PALMA doesnt work
model1_name = 'unet_100ep_floodnet-ps1024_randomsplit_lr5e-5_weightedCE_b2_ownNorm'
model2_name = 'segformer_100ep_floodnet-ps1024_randomsplit_lr3e-5_weightedCE_b2_ownNorm'
norm_dataset = 'floodnet'


# load model 1
MODEL1_PATH = os.path.join(models_dir,model1_name,model1_name+'_best.pt')
checkpoint1 = torch.load(MODEL1_PATH)
model1 = checkpoint1['model']
model1_label = 'U-Net' + '-' + label_add

# load model 2
MODEL2_PATH = os.path.join(models_dir,model2_name,model2_name+'_best.pt')
checkpoint2 = torch.load(MODEL2_PATH)
model2 = checkpoint2['model']
model2_label = 'SegFormer' + '-' + label_add

In [3]:
# load dataset and create data loader
from utils import make_loader, load_datasets, inverse_transform, augmentation, train_id_to_color, DataLoader
test_dataset = load_datasets(data_path, random_split=True, normalize = True, classes=dataset, patch_size=patch_size, only_test = True, dataset=norm_dataset)
test_loader = DataLoader(test_dataset)

In [7]:
try: 
    CLASSES = test_dataset.CLASSES
except:
    try:
        CLASSES = test_dataset.dataset.CLASSES
    except KeyError: 
        raise KeyError('dataset has no attribute CLASSES')
id_to_color, legend_elements = train_id_to_color(CLASSES)

In [12]:
# CLASSES = ['impervious', 'building', 'vegetation', 'tree', 'car', 'clutter']
dataSet = test_dataset


def calc_homogeneity(model, dataset, normalize = True, norm_dataset = 'potsdam'):
    try: 
        CLASSES = dataset.CLASSES
    except:
        try:
            CLASSES = dataset.dataset.CLASSES
        except KeyError: 
            raise KeyError('dataset has no attribute CLASSES')    
            
    id_to_color, legend_elements = train_id_to_color(CLASSES)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
    
    # array of values depending on number of classes
    mask_values = [i for i in range(len(CLASSES))]
    
    metrices = {
        "homogeneity" : [],
        "completeness" : [],
        "v-measure" : [],
        "rand" : [],
        "ari" : [],
        "mutual-info" : [],
        "str_similarity" : [],
        "false_splits" : [],
        "false_merges" : [],
        "rand-error" : [],
        "rand-pre" : [],
        "rand-rec": [],
        # "cluster_count_gt" : [],
        # "cluster_count_pred" : []
    }
    
    scores = pd.DataFrame(metrices)
    class_scores = pd.DataFrame(mask_values)
    
    # count per class
    # TODO: do the ccolumns need to be initilized?
    class_scores['cluster_count_gt'] = cluster_count_gt = [0] * len(mask_values)
    class_scores['cluster_count_pred'] = cluster_count_pred = [0] * len(mask_values)
    class_scores['pixel_count_gt'] = pixel_count_gt = [0] * len(mask_values)
    class_scores['pixel_count_pred'] = pixel_count_pred = [0] * len(mask_values)
    class_scores['eroded'] = eroded = [0] * len(mask_values)
    class_scores['ppc_pred_by_gt'] = [0] * len(mask_values)
    class_scores['eroded_percent'] = [0] * len(mask_values)
    
    
    
    for i in range(len(dataset)):
        inputImage, gt = dataset[i]
        inputImage = inputImage.to(device)    
        
        # inference / apply model 
        y_pred = torch.argmax(model(inputImage.unsqueeze(0)), dim=1).squeeze(0)
        
        # detach
        label_class_predicted = y_pred.cpu().detach().numpy()    
        label_class = gt.cpu().detach().numpy()
        
        # plt.imshow(id_to_color[label_class_predicted])        
        homogeneity, completeness, v_measure = metrics.homogeneity_completeness_v_measure(label_class.flatten(), label_class_predicted.flatten())
        rand = metrics.rand_score(label_class.flatten(), label_class_predicted.flatten())
        ari = metrics.adjusted_rand_score(label_class.flatten(), label_class_predicted.flatten())
        mi = metrics.mutual_info_score(label_class.flatten(), label_class_predicted.flatten())
        sim = ssim(label_class, label_class_predicted, data_range = len(CLASSES))
        false_splits, false_merges = vari(label_class, label_class_predicted)
        error, precision, recall = adapted_rand_error(label_class, label_class_predicted)
        

        # each class individually
        for j, val in enumerate(mask_values):
            # count in ground truth
            mask_gt = np.where(label_class == val, 1, 0)
            pixel_count_gt[j] = mask_gt.sum()
            lw_gt, num_gt = count_clusters(mask_gt)
            cluster_count_gt[j] = num_gt
            # count in prediction
            mask_pr = np.where(label_class_predicted == val, 1, 0)
            pixel_count_pred[j] = mask_pr.sum()
            lw_pr, num_pr = count_clusters(mask_pr)
            cluster_count_pred[j] = num_pr
            # delate in prediction
            kernel = np.ones((5, 5), np.uint8) 
            img_dilation = cv2.erode(mask_pr.astype('uint8'), kernel, iterations=1)
            diff = np.where(mask_pr != img_dilation, 1, 0)
            eroded[j] = diff.sum()
            
            
        
        score = [homogeneity, completeness, v_measure, rand, ari, mi, sim, false_splits, false_merges, error, precision, recall]
        class_scores['cluster_count_gt'] += cluster_count_gt
        class_scores['cluster_count_pred'] += cluster_count_pred
        class_scores['pixel_count_gt'] += pixel_count_gt
        class_scores['pixel_count_pred'] += pixel_count_pred
        class_scores['eroded'] += eroded
        
        # print(class_scores)
        
        scores.loc[dataset.get_name(i)] = score
        
    scores.loc['mean'] = scores.mean()
    scores.loc['median'] = scores.median()
    
    class_scores.loc['sum'] = class_scores.sum()
    class_scores['pixel_per_cluster_gt'] = class_scores['pixel_count_gt']/class_scores['cluster_count_gt']
    class_scores['pixel_per_cluster_pred'] = class_scores['pixel_count_pred']/class_scores['cluster_count_pred']
    class_scores['ppc_pred_by_gt'] = class_scores['pixel_per_cluster_pred'] / class_scores['pixel_per_cluster_gt']
    class_scores['eroded_percent'] = class_scores['eroded'] / class_scores['pixel_count_gt']
    
        
    return scores, class_scores
        
        

# input rgb image   


# Homogeneity without own scores

In [13]:
# CLASSES = ['impervious', 'building', 'vegetation', 'tree', 'car', 'clutter']
dataSet = test_dataset


def calc_homogeneity(model, dataset, normalize = True, norm_dataset = 'potsdam'):
    try: 
        CLASSES = dataset.CLASSES
    except:
        try:
            CLASSES = dataset.dataset.CLASSES
        except KeyError: 
            raise KeyError('dataset has no attribute CLASSES')    
            
    id_to_color, legend_elements = train_id_to_color(CLASSES)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
    
    # array of values depending on number of classes
    mask_values = [i for i in range(len(CLASSES))]
    
    metrices = {
        "homogeneity" : [],
        "completeness" : [],
        "v-measure" : [],
        "rand" : [],
        "ari" : [],
        "mutual-info" : [],
        "str_similarity" : [],
        "false_splits" : [],
        "false_merges" : [],
        "vi": [],
        "rand-error" : [],
        "rand-pre" : [],
        "rand-rec": [],
        # "cluster_count_gt" : [],
        # "cluster_count_pred" : []
    }
    
    scores = pd.DataFrame(metrices)
    class_scores = pd.DataFrame(mask_values)
    
    # count per class
    # TODO: do the ccolumns need to be initilized?
    class_scores['cluster_count_gt'] = cluster_count_gt = [0] * len(mask_values)
    class_scores['cluster_count_pred'] = cluster_count_pred = [0] * len(mask_values)
    class_scores['pixel_count_gt'] = pixel_count_gt = [0] * len(mask_values)
    class_scores['pixel_count_pred'] = pixel_count_pred = [0] * len(mask_values)
    class_scores['eroded'] = eroded = [0] * len(mask_values)
    class_scores['ppc_pred_by_gt'] = [0] * len(mask_values)
    class_scores['eroded_percent'] = [0] * len(mask_values)
    # class_scores['rand_error'] = c_error = [0] * len(mask_values)
    # class_scores['r_precision'] = c_precision = [0] * len(mask_values)
    # class_scores['r_recal'] = c_recall = [0] * len(mask_values)
    class_scores['false_splits'] = c_splits = [0] * len(mask_values)
    class_scores['false_merges'] = c_merges = [0] * len(mask_values)
    class_scores['vi'] = c_vi = [0] * len(mask_values)
    
    
    for i in tqdm(range(len(dataset))):
        inputImage, gt = dataset[i]
        inputImage = inputImage.to(device)    
        
        # inference / apply model 
        y_pred = torch.argmax(model(inputImage.unsqueeze(0)), dim=1).squeeze(0)
        
        # detach
        label_class_predicted = y_pred.cpu().detach().numpy()    
        label_class = gt.cpu().detach().numpy()
            
        # plt.imshow(id_to_color[label_class_predicted])        
        homogeneity, completeness, v_measure = metrics.homogeneity_completeness_v_measure(label_class.flatten(), label_class_predicted.flatten())
        rand = metrics.rand_score(label_class.flatten(), label_class_predicted.flatten())
        ari = metrics.adjusted_rand_score(label_class.flatten(), label_class_predicted.flatten())
        mi = metrics.mutual_info_score(label_class.flatten(), label_class_predicted.flatten())
        sim = ssim(label_class, label_class_predicted, data_range = len(CLASSES))
        false_splits, false_merges = vari(label_class, label_class_predicted)
        vi = false_splits + false_merges
        error, precision, recall = adapted_rand_error(label_class, label_class_predicted)
        

        # each class individually
        for j, val in enumerate(mask_values):
            # count in ground truth
            mask_gt = np.where(label_class == val, 1, 0)
            gt_sum = mask_gt.sum()
            pixel_count_gt[j] = gt_sum
            lw_gt, num_gt = count_clusters(mask_gt)
            cluster_count_gt[j] = num_gt
            # count in prediction
            mask_pr = np.where(label_class_predicted == val, 1, 0)
            pr_sum = mask_pr.sum()
            pixel_count_pred[j] = pr_sum
            lw_pr, num_pr = count_clusters(mask_pr)
            cluster_count_pred[j] = num_pr
            # delate in prediction
            kernel = np.ones((5, 5), np.uint8) 
            img_dilation = cv2.erode(mask_pr.astype('uint8'), kernel, iterations=1)
            diff = np.where(mask_pr != img_dilation, 1, 0)
            eroded[j] = diff.sum()
            
            c_splits[j], c_merges[j] = vari(mask_gt, mask_pr)
            c_vi[j] = c_splits[j] + c_merges[j]
            
#             if gt_sum > 0 and pr_sum > 0:
#                 c_error[j], c_precision[j], c_recall[j] = adapted_rand_error(mask_gt, mask_pr)
#                 c_splits[j], c_merges[j] = vari(mask_gt, mask_pr)
#             else:
#                 c_error[j], c_precision[j], c_recall[j] = (0,0,0)
#                 c_splits[j], c_merges[j] = (0, 0)
            
            
            
        
        score = [homogeneity, completeness, v_measure, rand, ari, mi, sim, false_splits, false_merges, vi, error, precision, recall]
        class_scores['cluster_count_gt'] += cluster_count_gt
        class_scores['cluster_count_pred'] += cluster_count_pred
        class_scores['pixel_count_gt'] += pixel_count_gt
        class_scores['pixel_count_pred'] += pixel_count_pred
        class_scores['eroded'] += eroded
        # class_scores['rand_error'] += c_error
        # class_scores['r_precision'] += c_precision
        # class_scores['r_recal'] += c_recall
        class_scores['false_splits'] += c_splits
        class_scores['false_merges'] += c_merges
        class_scores['vi'] += c_vi
        
        # print(class_scores)
        
        scores.loc[dataset.get_name(i)] = score
        
    class_scores = class_scores/len(dataset)
        
    scores.loc['mean'] = scores.mean()
    scores.loc['median'] = scores.median()
    
    # class_scores = class_scores/len(dataset)
    
    class_scores.loc['sum'] = class_scores.sum()
    class_scores['pixel_per_cluster_gt'] = class_scores['pixel_count_gt']/class_scores['cluster_count_gt']
    class_scores['pixel_per_cluster_pred'] = class_scores['pixel_count_pred']/class_scores['cluster_count_pred']
    class_scores['ppc_pred_by_gt'] = class_scores['pixel_per_cluster_pred'] / class_scores['pixel_per_cluster_gt']
    class_scores['eroded_percent'] = class_scores['eroded'] / class_scores['pixel_count_gt']
    
        
    return scores, class_scores
        
        

# input rgb image   


In [16]:
scores2.to_csv(('../results/test_homogeneity_'+model2_label+'_'+model2_name+'_'+norm_dataset+'Norm'+'.csv'))
class_scores2.to_csv(('../results/test_classscores_'+model2_label+'_'+model2_name+'_'+norm_dataset+'Norm'+'.csv'))

In [17]:
# scores1 = calc_homogeneity(model1, test_dataset)
scores1, class_scores1 = calc_homogeneity(model1, test_dataset)
scores1.to_csv(('../results/test_homogeneity_'+model1_label+'_'+model1_name+'_'+norm_dataset+'Norm'+'.csv'))
class_scores1.to_csv(('../results/test_classscores_'+model1_label+'_'+model1_name+'_'+norm_dataset+'Norm'+'.csv'))

 15%|█▌        | 904/5960 [18:49<1:43:36,  1.23s/it]/home/j/j_sten07/.local/lib/python3.10/site-packages/skimage/metrics/_adapted_rand_error.py:91: RuntimeWarning: invalid value encountered in scalar divide
  precision = sum_p_ij2 / sum_a2
/home/j/j_sten07/.local/lib/python3.10/site-packages/skimage/metrics/_adapted_rand_error.py:92: RuntimeWarning: invalid value encountered in scalar divide
  recall = sum_p_ij2 / sum_b2
/home/j/j_sten07/.local/lib/python3.10/site-packages/skimage/metrics/_adapted_rand_error.py:94: RuntimeWarning: invalid value encountered in scalar divide
  fscore = sum_p_ij2 / (alpha * sum_a2 + (1 - alpha) * sum_b2)
 57%|█████▋    | 3383/5960 [1:10:16<51:28,  1.20s/it]  /home/j/j_sten07/.local/lib/python3.10/site-packages/skimage/metrics/_adapted_rand_error.py:91: RuntimeWarning: invalid value encountered in scalar divide
  precision = sum_p_ij2 / sum_a2
/home/j/j_sten07/.local/lib/python3.10/site-packages/skimage/metrics/_adapted_rand_error.py:92: RuntimeWarning: in

In [18]:
print(scores1.iloc[-2:])
print(scores2.iloc[-2:])
# print(scores2)
print(class_scores1)
print(class_scores2)
# print(clusters_segformer['cluster_count_pred']/clusters_segformer['pixel_count_pred'])
# print(clusters_segformer['cluster_count_gt']/clusters_segformer['pixel_count_gt'])
# print(clusters_unet['cluster_count_pred']/clusters_unet['pixel_count_pred'])
# print(clusters_unet['cluster_count_gt']/clusters_unet['pixel_count_gt'])
# print(clusters2['pixel_per_cluster_pred']/clusters2['pixel_per_cluster_gt'])
# print(clusters1['pixel_per_cluster_pred']/clusters1['pixel_per_cluster_gt'])

        homogeneity  completeness  v-measure      rand       ari  mutual-info  \
mean       0.678974      0.384547   0.405903  0.845024  0.447668     0.312674   
median     0.694527      0.442057   0.482438  0.871749  0.531084     0.292531   

        str_similarity  false_splits  false_merges        vi  rand-error  \
mean          0.857554      0.521940      0.280541  0.802481    0.147929   
median        0.909080      0.439338      0.213240  0.702266    0.112353   

        rand-pre  rand-rec  
mean    0.817670  0.906834  
median  0.862645  0.951828  
        homogeneity  completeness  v-measure      rand       ari  mutual-info  \
mean       0.584987      0.679370   0.559393  0.869068  0.586936     0.262634   
median     0.607879      0.697605   0.594221  0.896980  0.651091     0.208181   

        str_similarity  false_splits  false_merges        vi  rand-error  \
mean          0.916018      0.277728      0.352734  0.630461    0.121355   
median        0.962814      0.167374      0.

In [19]:
class_scores1

,0,cluster_count_gt,cluster_count_pred,pixel_count_gt,pixel_count_pred,eroded,ppc_pred_by_gt,eroded_percent,false_splits,false_merges,vi,pixel_per_cluster_gt,pixel_per_cluster_pred
0,0.000000,0.195973,420.087248,2.430522e+04,4.044354e+03,2039.161913,0.000078,0.083898,0.020841,0.024688,0.045529,124023.218322,9.627415
1,0.000168,0.392617,260.848658,2.685721e+04,1.682373e+03,1111.229698,0.000094,0.041375,0.009112,0.075189,0.084301,68405.547009,6.449615
2,0.000336,0.304698,773.197148,2.860674e+04,5.921400e+04,7482.972483,0.000816,0.261581,0.131597,0.032299,0.163896,93885.567181,76.583308
3,0.000503,0.371812,1005.721980,3.942144e+04,1.107039e+04,4754.769966,0.000104,0.120614,0.041102,0.060908,0.102011,106025.168773,11.007406
4,0.000671,0.456879,993.519799,5.829768e+04,7.470108e+04,8655.367114,0.000589,0.148468,0.121281,0.056404,0.177685,127599.765332,75.188316
5,0.000839,1.220973,1870.378356,1.255159e+05,1.597910e+05,13876.467114,0.000831,0.110555,0.174039,0.068235,0.242273,102799.850076,85.432451
6,0.001007,1.459564,2381.008725,1.962137e+05,2.099710e+05,16083.859732,0.000656,0.081971,0.215659,0.147408,0.363068,134433.124727,88.185718
7,0.001174,0.267953,64.904195,2.108688e+03,3.795877e+03,745.541107,0.007432,0.353557,0.015659,0.004696,0.020356,7869.618660,58.484310
8,0.001342,0.080034,64.098993,2.184802e+03,2.827827e+03,412.787248,0.001616,0.188936,0.009686,0.005156,0.014842,27298.574423,44.116551
9,0.001510,4.432383,3323.296644,5.450646e+05,5.214781e+05,31903.315772,0.001276,0.058531,0.340165,0.241209,0.581373,122973.285157,156.915914


In [20]:
class_scores2

,0,cluster_count_gt,cluster_count_pred,pixel_count_gt,pixel_count_pred,eroded,ppc_pred_by_gt,eroded_percent,false_splits,false_merges,vi,pixel_per_cluster_gt,pixel_per_cluster_pred
0,0.000000,0.195973,2.547987,2.430522e+04,2.680561e+04,438.202852,0.084825,0.018029,0.015592,0.020935,0.036527,124023.218322,10520.312064
1,0.000168,0.392617,1.617282,2.685721e+04,3.449817e+04,942.008389,0.311831,0.035075,0.042408,0.036473,0.078881,68405.547009,21330.955078
2,0.000336,0.304698,3.521980,2.860674e+04,4.111640e+04,1182.273993,0.124345,0.041329,0.053244,0.036434,0.089678,93885.567181,11674.229765
3,0.000503,0.371812,2.345470,3.942144e+04,5.012244e+04,1067.708893,0.201555,0.027084,0.045355,0.042674,0.088029,106025.168773,21369.895558
4,0.000671,0.456879,2.924161,5.829768e+04,5.048672e+04,1224.251007,0.135309,0.021000,0.038038,0.067655,0.105693,127599.765332,17265.370266
5,0.000839,1.220973,7.912584,1.255159e+05,9.773892e+04,1940.446309,0.120159,0.015460,0.060691,0.082511,0.143202,102799.850076,12352.339257
6,0.001007,1.459564,10.232215,1.962137e+05,1.858218e+05,3624.473154,0.135089,0.018472,0.115822,0.169999,0.285821,134433.124727,18160.466352
7,0.001174,0.267953,0.608893,2.108688e+03,3.453546e+03,289.368289,0.720727,0.137227,0.013179,0.006012,0.019191,7869.618660,5671.847065
8,0.001342,0.080034,0.493624,2.184802e+03,4.630095e+03,221.756376,0.343600,0.101500,0.014685,0.005058,0.019743,27298.574423,9379.798776
9,0.001510,4.432383,25.348154,5.450646e+05,5.539023e+05,6143.285570,0.177695,0.011271,0.198071,0.271711,0.469782,122973.285157,21851.780089
